In [1]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

In [8]:
#viz

ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = BertModel.from_pretrained(ckpt)
text = 'time flies like an arrow'

show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
#tokenization

inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids.shape

torch.Size([1, 5])

In [17]:
# embeddings

from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

In [21]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.shape

torch.Size([1, 5, 768])

In [46]:
# q, k , v

import torch 
from math import sqrt
import torch.nn.functional as F

def scaled_dot_product_attention(key, query, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(self.q(hidden_state), self.k(hidden_state), self.v(hidden_state))
        return attn_outputs